
# Project Overview

Tanzania, with a population of over 57 million, faces significant challenges in providing access to clean water, a fundamental need for its citizens. The country has numerous water points spread across its vast landscape, but many of these wells have deteriorated over time. Some are fully functional, others require repair, and some have failed completely. This project focuses on building a classifier to predict the condition of these water wells, leveraging various data points such as the type of pump used, installation dates, geographic location, and other relevant factors. The outcome of this model will be crucial for organizations like NGOs and the Tanzanian government to prioritize and allocate resources effectively, ensuring better water access for the population.

# Business Understanding

Ensuring the functionality of water wells is critical to providing safe and clean water to communities, particularly in a developing country like Tanzania. The failure of these water points can have severe consequences, leading to water scarcity, poor hygiene, and health issues. For stakeholders such as the Tanzanian government and NGOs, understanding the factors that contribute to the deterioration of wells can drive strategic decisions, such as where to allocate repair resources or how to design more durable wells in the future. This project aims to address this need by developing a predictive model that can classify the condition of wells, enabling these stakeholders to act proactively in maintaining water accessibility.

# Objectives

The overarching goal of this project is to develop a robust and accurate classification model that can predict the condition of water wells in Tanzania. This model will assist in identifying wells that are likely to fail, enabling stakeholders to take preventive measures. By understanding the key factors influencing well functionality, the project will also provide insights that can be used to improve the design and maintenance of future water points. Ultimately, this project seeks to contribute to a more reliable water supply infrastructure in Tanzania, improving the quality of life for its residents.

## Main Objectives

The main objectives of this project include:

1. Analyzing the data to identify the most significant factors contributing to the condition of the wells.
2. Building a predictive model that can classify water wells into one of three categories: functional, in need of repair, or non-functional.
3. Providing actionable insights and recommendations to stakeholders, such as NGOs and the Tanzanian government, to guide their efforts in maintaining and improving water well infrastructure.

## Specific Objectives

To achieve the main objectives, the project will focus on the following specific objectives:

1. Conducting exploratory data analysis (EDA) to uncover patterns and relationships within the data.
2. Developing and fine-tuning a machine learning model for classification to predict well conditions.
3. Evaluating the model's performance using appropriate metrics to ensure accuracy and reliability.
4. Interpreting the model's results to provide clear, actionable recommendations for stakeholders.

# Data Understanding

The dataset for this project comprises detailed information on water wells across Tanzania, including variables such as; the type of pump installed, the year of installation, and the geographic coordinates of each well. Additional features may include the depth of the well, the source of water, and the materials used in construction. Understanding the distribution of these features and their potential impact on well functionality is crucial for building an effective classification model. 

#### Importing the necessary datasets

In [ ]:
#Importing the necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import category_encoders as ce
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from category_encoders import WOEEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV


#### Loading the Datasets

In [ ]:
# Import the necessary datasets
df_training_labels = pd.read_csv('Data\Training_Labels.csv', index_col= 0)
df_training_values = pd.read_csv('Data\Training_values.csv', index_col = 0)
df_test_values = pd.read_csv('Data\Test_set.csv', index_col = 0)

#### Inspecting the data

In [ ]:
# Getting a preview of the first 5 rows of the training labels data
df_training_labels.head()

In [ ]:
# Getting a preview of the first 5 rows of the training values data
df_training_values.head()

#### Merging the data

In [ ]:
# merging the training values and training labels dataset
df_training_set = df_training_values.merge(df_training_labels, on = "id")

# Getting a preview of the first 5 rows of the merged data
df_training_set.head()

#### Inspecting the merged data

In [ ]:
# Getting a summary of the dataset
df_training_set.info()

In [ ]:
# Checking the shape of the dataset
df_training_set.shape

In [ ]:
# Getting the statistics of numerical columns
df_training_set.describe()

# Data Cleaning

#### Dropping columns with information that is not relevant to prediction

In [ ]:
# Identify columns with similar information
similar_columns = {
    'extraction_type': ['extraction_type_group', 'extraction_type_class'],
    'management': ['management_group'],
    'payment': ['payment_type'],
    'water_quality': ['quality_group'],
    'quantity': ['quantity_group'],
    'source': ['source_type', 'source_class'],
    'waterpoint_type': ['waterpoint_type_group']
}

# Drop the redundant columns, retaining only the first column of each group
columns_to_drop = [col for group in similar_columns.values() for col in group]
df_training_set = df_training_set.drop(columns=columns_to_drop)

# Dropping columns with information that is not relevant for prediction
df_training_set = df_training_set.drop(columns=['wpt_name','subvillage','num_private', 'recorded_by'])

# Display the shape of the resultant dataframe
df_training_set.shape

#### Handling Missing Data 

In [ ]:
# Checking for missing values
total_values = 59400
missing_values = df_training_set.isnull().sum()

# Getting the percentage of missing values in each column
missing_percentage = (missing_values/total_values)*100
missing_percentage

In [ ]:
#Dropping columns where the percentange of missing values is more than 20
columns_to_drop = missing_percentage[missing_percentage > 20].index
df_training_set = df_training_set.drop(columns=columns_to_drop)

# Impute specific columns with 'Unknown'
df_training_set[['scheme_management', 'funder', 'installer']] = df_training_set[['scheme_management', 'funder', 'installer']].fillna('Unknown')

# Create imputers for numerical and categorical features
numerical_imputer = SimpleImputer(strategy='median')  # For numerical columns
categorical_imputer = SimpleImputer(strategy='most_frequent')  # For categorical columns

# Impute numerical features
df_training_set[['population']] = numerical_imputer.fit_transform(df_training_set[['population']])

# Impute categorical features
df_training_set[['permit', 'public_meeting']] = categorical_imputer.fit_transform(df_training_set[['permit', 'public_meeting']])

# Checking for missing values
df_training_set.isnull().sum()

#### Checking for duplicates

In [ ]:
# Identify duplicated index values
duplicated_index = df_training_set.index.duplicated()

# Display the rows with duplicated index values
duplicate_rows = df_training_set[duplicated_index]
duplicate_rows

#### Converting Datatypes

In [ ]:
# Checking data types
df_training_set.dtypes

In [ ]:
# Fixing the datatypes
df_training_set['date_recorded'] = pd.to_datetime(df_training_set['date_recorded'], format='%d/%m/%Y')

#### Fixing Spelling mistakes in the installer column

In [ ]:
df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('District Water Department', 'District water depar', 'Distric Water Department'),value='District water department')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('FinW', 'Fini water', 'FINI WATER'), value='Fini Water')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace='JAICA',value='Jaica')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('COUN', 'District COUNCIL', 'DISTRICT COUNCIL', 'District Counci','District Council', 'Council', 'Counc', 'District  Council', 'Distri'),value='District council')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('RC CHURCH', 'RC Churc', 'RC', 'RC Ch', 'RC C', 'RC CH', 'RC church','RC CATHORIC'),value='RC Church')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('Central Government', 'Tanzania Government','central government', 'Cental Government', 'Cebtral Government','Tanzanian Government', 'Tanzania government', 'Centra Government','CENTRAL GOVERNMENT', 'TANZANIAN GOVERNMENT', 'Central govt', 'Centr','Centra govt'), value='Central government')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('World vision', 'World Division', 'World Vision'),value='world vision')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('Unisef', 'UNICEF'),value='Unicef')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace='DANID',value='DANIDA')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('villigers', 'villager', 'Villagers', 'Villa', 'Village', 'Villi','Village Council', 'Village Counil', 'Villages', 'Vill', 'Village community', 'Villaers', 'Village Community', 'Villag', 'Villege Council', 'Village council','Village  Council', 'Villagerd', 'Villager', 'Village Technician','Village Office', 'Village community members'),value='villagers')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('Commu', 'Communit', 'commu', 'COMMU', 'COMMUNITY'),value='Community')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('GOVERNMENT', 'GOVER', 'GOVERNME', 'GOVERM', 'GOVERN', 'Gover', 'Gove','Governme', 'Governmen'),value='Government')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace='Hesawa',value='HESAWA')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('Colonial Government'),value='Colonial government')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('Government of Misri'),value='Misri Government')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('Italy government'),value='Italian government')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('British colonial government'),value='British government')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('Concern /government'),value='Concern/Government')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('Village Government'),value='Village government')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('Government and Community'),value='Government /Community')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('Cetral government /RC'),value='RC church/Central Gover')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('Government /TCRS', 'Government/TCRS'),value='TCRS /Government')

df_training_set['installer'] = df_training_set['installer'].replace(to_replace=('ADRA /Government'),value='ADRA/Government')

In [ ]:
df_training_set["installer"].value_counts()

#### Looking into the population column

In [ ]:
df_training_set["population"].value_counts()

In [ ]:
df_training_set.loc[df_training_set['population']==0].groupby('status_group')['population'].count()

Population of zero where the are functional wells indicates missing values

In [ ]:
df_training_set[df_training_set['population']!= 0].describe()

The median of the population column without zeros is 150. We shall impute the zeros with the median

In [ ]:
# Changing zeros to the median
df_training_set['population'] = df_training_set['population'].replace(to_replace = 0 , value = 150) 
# Checking the value counts
df_training_set['population'].value_counts()

#### Feature engineering
Creating a decade column from the construction year decade

In [ ]:
# Checking the value types of the construction column
df_training_set['construction_year'].value_counts()

In [ ]:
# Imputing the zeros in the construction year column  with the median

# Calculate the median of the non-zero values in the 'construction_year' column
median_year = df_training_set['construction_year'].replace(0, np.nan).median()

# Impute zeros with the calculated median
df_training_set['construction_year'] = df_training_set['construction_year'].replace(0, median_year)

In [ ]:
# Checking the value counts for the construction year column
df_training_set['construction_year'].value_counts()

In [ ]:
 # Creating the decade column
df_training_set['decade'] = df_training_set['construction_year']

In [ ]:
df_training_set["decade"] = df_training_set['decade'].replace(to_replace = (1960,1961,1962,1963,1964,1965,1966,1967,1968,1969),
                        value ='60s')
df_training_set["decade"] = df_training_set['decade'].replace(to_replace = (1970,1971,1972,1973,1974,1975,1976,1977,1978,1979),
                        value ='70s')
df_training_set["decade"] = df_training_set['decade'].replace(to_replace = (1980,1981,1982,1983,1984,1985,1986,1987,1988,1989),
                        value ='80s')
df_training_set["decade"] = df_training_set['decade'].replace(to_replace = (1990,1991,1992,1993,1994,1995,1996,1997,1998,1999),
                        value ='90s')
df_training_set["decade"] = df_training_set['decade'].replace(to_replace = (2000,2001,2002,2003,2004,2005,2006,2007,2008,2009),
                        value ='00s')
df_training_set["decade"] = df_training_set['decade'].replace(to_replace = (2010,2011,2012,2013),
                        value ='10s')

In [ ]:
df_training_set['decade'].value_counts() 

In [ ]:
# Dropping unnecessary columns
df_training_set = df_training_set.drop(columns = ["date_recorded"])

In [ ]:
# Preview of the resultant dataframe
train_data = df_training_set
train_data.head()

In [ ]:
train_data.to_csv('cleaned_data.csv', index=False)

# Exploratory Data Analysis

## Univariate analysis

### Distribution of Well Conditions

In [ ]:
plt.figure(figsize=(8, 6))
status_counts = train_data['status_group'].value_counts()
plt.pie(status_counts, labels=status_counts.index, autopct='%1.1f%%', colors=plt.get_cmap('Set2').colors)
plt.title('Percentage Distribution of Water Well Status')
plt.show()

54.3% of the Water wells are functional, 38.4% are non functional and 7.3% need repair.

### Distribution of Water Quality

In [ ]:
sns.countplot(x='water_quality', data= train_data)
plt.title('Distribution of Water Quality')
plt.xticks(rotation=45, ha='right')
plt.show()

### Top Ten Funders

In [ ]:
plt.figure(figsize=(10, 6))
train_data['funder'].value_counts().head(10).plot(kind='bar')
plt.title('Top 10 Funders')
plt.xlabel('Funder')
plt.ylabel('Count')
plt.xticks(rotation =45 , ha='right')
plt.show()

### Pump Type distribution

In [ ]:
plt.figure(figsize=(26,10))
sns.countplot(x='extraction_type', data= train_data)
plt.title('Distribution of Pump Types')
plt.show()

### Geographical distribution of wells

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(train_data['longitude'], train_data['latitude'], c= train_data['status_group'].astype('category').cat.codes, alpha=0.5)
plt.title('Geographical Distribution of Wells')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.colorbar(label='Status Group')
plt.show()

## Bivariate Analysis

### Pump Type vs Well Condition

In [ ]:
plt.figure(figsize=(26,12))
sns.countplot(x='extraction_type', hue='status_group', data= train_data)
plt.title('Pump Type vs Well Condition')
plt.show()

#### Water Quality VS Well Condition

In [ ]:
plt.figure(figsize=(26,12))
sns.countplot(x='water_quality', hue='status_group', data= train_data)
plt.title('Water Quality vs Well Condition')
plt.show()

#### Region Vs Well Condition

In [ ]:
plt.figure(figsize=(26,10))
sns.countplot(x='region', hue='status_group', data= train_data)
plt.title('Region vs Well Condition')
plt.show()

## Multivariate Analysis

In [ ]:
correlation_matrix = train_data[['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Numerical Features')
plt.show()

In [ ]:
train_data.dtypes

# Modelling

## Data preprocessing

In [ ]:
# Creating a copy of the original data to preserve it
data_df = train_data.copy()

#### Converting the Target variable to binary

In [ ]:
# Checking the value counts of the target variable
data_df['status_group'].value_counts()

Converting the target variable to binary where functional and functional needs repair are 1 and non functional is 0.

In [ ]:
# Creating a column for the binary target. 
data_df['binary_target'] = data_df['status_group'].apply(lambda x: 1 if x in ['functional', 'functional needs repair'] else 0)
# Checking the value_counts
data_df['binary_target'].value_counts()

#### Converting the permit and public meeting columns to boolean values

In [ ]:
# Converting the permit and public meeting columns to boolean values
data_df[['permit','public_meeting']] = data_df[['permit','public_meeting']].astype(bool).astype(int) 

#### Selecting Features for the model

In [ ]:
# Select features for the model
features = ['gps_height', 'longitude', 'latitude', 'basin', 'region', 
                'population', 'public_meeting', 'scheme_management', 'permit',
                'extraction_type', 'management', 'water_quality', 'quantity', 
                'source',]
    
X = data_df[features]
y = data_df['binary_target']

#### Scaling numerical columns and encoding categorical columns

In [ ]:
# For this process we use a Robust Scaler() and WoE encoder

In [ ]:
 # Identify numeric and categorical columns
numeric_features = X.select_dtypes(include=['int32','int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

In [ ]:
def preprocess_data(numeric_features, categorical_features):
    # Create preprocessing steps for numeric features
    numeric_transformer = Pipeline(steps=[('scaler', RobustScaler())])

    # Create preprocessing steps for categorical features using WoE Encoder
    categorical_transformer = Pipeline(steps=[('woe', WOEEncoder(cols=categorical_features))])

    # Combine preprocessing for numeric and categorical features
    preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
    
    return preprocessor

# Use the preprocessor
preprocessor = preprocess_data(numeric_features, categorical_features)


#### Splitting the data into the training and test set

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Fit and transform the training data
X_train_preprocessed = preprocessor.fit_transform(X_train, y_train)

# Transform the test data
X_test_preprocessed = preprocessor.transform(X_test)

## Model Training

### Logistic Regression model

In [ ]:
# Logistic Regression
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train_preprocessed, y_train)
y_pred_lr = lr_model.predict(X_test_preprocessed)

# Evaluation
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

### Decision Tree Classifier

In [ ]:
# Decision Tree
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train_preprocessed, y_train)
y_pred_dt = dt_model.predict(X_test_preprocessed)

# Evaluation
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))
print(classification_report(y_test, y_pred_dt))

### EXtra Trees Classifier

In [ ]:
# Extra Trees
et_model = ExtraTreesClassifier(random_state=42)
et_model.fit(X_train_preprocessed, y_train)
y_pred_et = et_model.predict(X_test_preprocessed)

# Evaluation
print("Extra Trees Accuracy:", accuracy_score(y_test, y_pred_et))
print(classification_report(y_test, y_pred_et))

### Random Forest Classifier

In [ ]:
# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_preprocessed, y_train)
y_pred_rf = rf_model.predict(X_test_preprocessed)

# Evaluation
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

In [ ]:
# Random Forest with GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_preprocessed, y_train)

# Best model from GridSearch
best_rf_model = grid_search.best_estimator_
y_pred_best_rf = best_rf_model.predict(X_test_preprocessed)

# Evaluation
print("Random Forest with GridSearchCV Accuracy:", accuracy_score(y_test, y_pred_best_rf))
print(classification_report(y_test, y_pred_best_rf))
print("Best Parameters found by GridSearch:", grid_search.best_params_)

### XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train_preprocessed, y_train)
y_pred_xgb = xgb_model.predict(X_test_preprocessed)

# Evaluation
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))

### K-Nearest Neighbors

In [ ]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train_preprocessed, y_train)
y_pred_knn = knn_model.predict(X_test_preprocessed)

# Evaluation
print("K-Nearest Neighbors Accuracy:", accuracy_score(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))

In [ ]:
# LightGBM
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train_preprocessed, y_train)
y_pred_lgb = lgb_model.predict(X_test_preprocessed)

# Evaluation
print("LightGBM Accuracy:", accuracy_score(y_test, y_pred_lgb))
print(classification_report(y_test, y_pred_lgb))

### Model Selection 

In [ ]:
# Initialize an empty list to store the results
results = []

# Define a function to evaluate a model and store the results
def evaluate_model(model_name, model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else None

    results.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc
    })

# Creating a dictionary with the models
models = {
    'Logistic Regression': lr_model,
    'Decision Tree': dt_model,
    'Extra Trees': et_model,
    'Random Forest': rf_model,
       'XGBoost': xgb_model,
    'K-Nearest Neighbors': knn_model,
    'LightGBM': lgb_model
}

# Evaluate each model and store the results
for model_name, model in models.items():
    evaluate_model(model_name, model, X_test_preprocessed, y_test)

# Create a DataFrame with the results
results_df = pd.DataFrame(results)

# Display the results DataFrame
results_df

Random Forest is considered the best model in this context because it performs exceptionally well across several key evaluation metrics:

1. High Accuracy (0.848822):
Random Forest has the highest accuracy among all the models, indicating that it consistently makes the correct predictions for both classes (functional and non-functional wells).

2. Balanced Precision (0.855246) and Recall (0.907909):
A precision of 0.855246 suggests that the model has a low rate of false positives, meaning it's good at not falsely predicting wells as functional when they are not.
With a recall of 0.907909, Random Forest is excellent at detecting nearly all functional wells, minimizing false negatives.

3. High F1-Score (0.880791):
A high F1-score of 0.880791 suggests that Random Forest excels in both accurately predicting functional wells and identifying them correctly.

4. Top ROC-AUC (0.910436):
. A ROC-AUC of 0.910436 indicates that Random Forest is highly effective at separating the two classes, meaning it is very reliable in distinguishing between functional and non-functional wells.


In [ ]:
### Hyperparameter tuning of selected model